In [14]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split

## 1. Temporal partitioning

In [13]:
trn_crd = pd.read_csv('LWAT01_trncrd.csv', header=None, usecols=[0, 1])
trn_crd.head()

,0,1
0,0.076851,0.894201
1,0.076847,0.894169
2,0.076953,0.894230
3,0.076847,0.894158
4,0.076983,0.894148


In [9]:
trn_rss = pd.read_csv('LWAT01_trnrss.csv', header=None)
trn_rss.head()

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
0,-200,-200,-200,-200,-99,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
1,-200,-200,-200,-200,-95,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
2,-200,-200,-200,-200,-107,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
3,-200,-200,-200,-200,-90,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
4,-200,-200,-200,-103,-98,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200


In [14]:
tst_crd = pd.read_csv('LWAT01_tstcrd.csv', header=None, usecols=[0, 1])
tst_crd.head()

,0,1
0,0.076948,0.894150
1,0.076856,0.894155
2,0.076833,0.894133
3,0.076775,0.894027
4,0.076723,0.893950


In [11]:
tst_rss = pd.read_csv('LWAT01_tstrss.csv', header=None)
tst_rss.head()

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
0,-200,-200,-200,-115,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
1,-200,-200,-200,-200,-91,-109,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
2,-200,-200,-200,-200,-96,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
3,-200,-200,-200,-200,-100,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
4,-200,-200,-200,-200,-109,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200


### 1.1 KNN-based fingerprinting (K=3)

In [9]:
# Función para calcular la distancia euclidiana entre dos puntos
def haversine(coord1, coord2):
    # Radio de la Tierra en metros
    R = 6371000  
    
    lon1, lat1 = coord1
    lon2, lat2 = coord2
    
    # Diferencia de coordenadas
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 

    # Calcular la distancia usando la fórmula del haversine
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a)) 
    distance = R * c
    
    return distance

In [43]:
# Crear el modelo k-NN
knn_model = KNeighborsRegressor(n_neighbors=3)
knn_model.fit(trn_rss, trn_crd)

# Predecir los RSSI para los datos de prueba
predicted_crd = knn_model.predict(tst_rss)

# Convertimos a array
tst_crd_array = np.array(tst_crd)

# Calcular la distancia euclidiana entre las coordenadas predichas y las coordenadas reales
distances = [haversine(pred, real) for pred, real in zip(predicted_crd, tst_crd_array)]

# Calcular el error medio
mean_error = np.mean(distances)

# Imprimir el error medio
print("Error medio:", mean_error)

Error medio: 504.44430155631073


## 2. Random partitioning

In [3]:
# Se ejecuta el script 'LoRaWAN_random_split.py' para generar ficheros particionados aleatoriamente que se cargan a continuación
train_rss_random = pd.read_csv('train_rss_random.csv')
train_crd_random = pd.read_csv('train_crd_random.csv')
val_rss_random = pd.read_csv('val_rss_random.csv')
val_crd_random = pd.read_csv('val_crd_random.csv')

In [4]:
train_rss_random.head()

,BS 1,BS 2,BS 3,BS 4,BS 5,BS 6,BS 7,BS 8,BS 9,BS 10,...,BS 59,BS 60,BS 61,BS 62,BS 63,BS 64,BS 65,BS 66,BS 67,BS 68
0,-103,-103,-200,-200,-200,-200,-200,-200,-200,-79,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
1,-111,-92,-85,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
2,-117,-200,-200,-200,-200,-200,-120,-109,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
3,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
4,-200,-98,-200,-200,-200,-200,-108,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200


In [5]:
train_crd_random.head()

,Longitude,Latitude
0,0.077119,0.893511
1,0.077238,0.893668
2,0.077709,0.893796
3,0.076969,0.893717
4,0.077164,0.893656


In [6]:
val_rss_random.head()

,BS 1,BS 2,BS 3,BS 4,BS 5,BS 6,BS 7,BS 8,BS 9,BS 10,...,BS 59,BS 60,BS 61,BS 62,BS 63,BS 64,BS 65,BS 66,BS 67,BS 68
0,-200,-93,-105,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
1,-200,-200,-200,-112,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
2,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
3,-200,-200,-200,-200,-104,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
4,-200,-200,-200,-200,-200,-200,-200,-200,-99,-200,...,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200


In [7]:
val_crd_random.head()

,Longitude,Latitude
0,0.077189,0.893774
1,0.076781,0.894037
2,0.076878,0.894083
3,0.077135,0.894164
4,0.077427,0.894000


### 2.1 KNN-based fingerprinting (K=3)

In [10]:
# Crear el modelo k-NN
knn_model = KNeighborsRegressor(n_neighbors=3)
knn_model.fit(train_rss_random, train_crd_random)

# Predecir los RSSI para los datos de prueba
predicted_crd_random = knn_model.predict(val_rss_random)

# Convertimos a array
val_crd_array_random = np.array(val_crd_random)

# Calcular la distancia euclidiana entre las coordenadas predichas y las coordenadas reales
distances_random = [haversine(pred, real) for pred, real in zip(predicted_crd_random, val_crd_array_random)]

# Calcular el error medio
mean_error_random = np.mean(distances_random)

# Imprimir el error medio
print("Error medio:", mean_error_random)

Error medio: 420.66445099883015
